In [1]:
# load credentials
import os
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [4]:
# pip install langchain_community tiktoken langchainhub chromadb langchain langchain-huggingface PyMuPDF

In [6]:
# load modules
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

from langchain_community.vectorstores import Chroma, FAISS, AstraDB
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [7]:
import os
from langchain_community.document_loaders import TextLoader

folder_path = "F:\\LLM_projects\\research_paper_bot\\web_scraping"

# Get all .txt files in the folder
txt_files = [file for file in os.listdir(folder_path) if file.endswith('.txt')]

all_pages = []  # To store pages from all files

for txt in txt_files:
    # print(f"Processing {txt} file...")
    loader = TextLoader(file_path=os.path.join(folder_path, txt), encoding="utf-8")
    
    # Load the documents for this file and append to `all_pages`
    for doc in loader.lazy_load():
        all_pages.append(doc)

# Output all collected pages
print(f"Total pages loaded: {len(all_pages)}")

    # txtfile = loader.load()
    # print(txtfile[0].page_content)

Total pages loaded: 59


In [51]:
# all_pages

In [38]:
# from langchain_community.document_loaders import PyMuPDFLoader

# loader = PyMuPDFLoader("F:\\LLM_projects\\research_paper_bot\\demo_3.pdf")

# page = []
# for doc in loader.lazy_load():
#     page.append(doc)
#     # if len(page) >= 10:
#     #     page = []

In [8]:
# Splitting document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 200,
        separators=[
        "\n"]
)

splits = text_splitter.split_documents(all_pages)
# splits

In [9]:
len(splits)

1174

# Embedding model: BAAI/bge-m3

In [22]:
# model_name = 'BAAI/bge-m3'

# # model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": True}


# vec_db = Chroma.from_documents(
#     documents=splits,
#     embedding= HuggingFaceBgeEmbeddings(model_name = model_name,
#                                         encode_kwargs = encode_kwargs)
#     )

model_name = "G:\\office\\langflow\\models--BAAI--bge-m3\\snapshots\\babcf60cae0a1f438d7ade582983d4ba462303c2"
# model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)


ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_KEYSPACE = os.getenv("ASTRA_DB_KEYSPACE")

# vec_db = AstraDB.from_documents(    
#     collection_name="pregnancy_bot",
#     api_endpoint=ASTRA_DB_API_ENDPOINT,
#     token=ASTRA_DB_APPLICATION_TOKEN,
#     namespace='default_keyspace',
    
#     documents=splits,
#     embedding= hf_embedding_model
#     )


# cleanup
# vec_db.delete_collection()

In [23]:
# as data is already loaded
from langchain_astradb import AstraDBVectorStore

vec_db = AstraDBVectorStore(
    embedding=hf_embedding_model,
    collection_name="pregnancy_bot",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace='default_keyspace',
)

APICommander about to raise from: [{'message': "Collection already exists: trying to create Collection ('pregnancy_bot') with different settings", 'errorCode': 'EXISTING_COLLECTION_DIFFERENT_SETTINGS', 'id': '8487252e-5ce9-4176-9699-0635672fec4b', 'title': 'Collection already exists', 'family': 'REQUEST', 'scope': 'EMPTY'}]
d:\torch-venv\lib\site-packages\langchain_astradb\utils\astradb.py:387: UserWarning: Astra DB collection 'pregnancy_bot' is detected as having indexing turned on for all fields (either created manually or by older versions of this plugin). This implies stricter limitations on the amount of text each string in a document can store. Consider indexing anew on a fresh collection to be able to store longer texts. See https://github.com/langchain-ai/langchain-datastax/blob/main/libs/astradb/README.md#warnings-about-indexing for more details.
  if not self._validate_indexing_policy(


## Vector Retriever

In [24]:
vec_retriever = vec_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# LLM

In [25]:
# os.environ["API_TOKEN"] = hf_token

from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    top_k=5,
    top_p=0.85,
    # typical_p=0.9,
    temperature=0.4,
    max_new_tokens=2048,
    do_sample=False,
    # repetition_penalty=1.1,
    stop_sequences=["\n\n"],
    huggingfacehub_api_token=HF_TOKEN
)



In [15]:
# Prompt setup
prompt = hub.pull("rlm/rag-prompt")


# format context
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": vec_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

d:\torch-venv\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [26]:
from langchain_core.prompts import PromptTemplate

template = """You are a doctor who is answering pregnancy questions.
Use the following pieces of context to answer the question at the end.
Always reply in bangla text, do not answer in english. No need to translate in english too.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
End the answer on a single । punctuation.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": vec_retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [27]:
print("Bot: আমি প্রেগনেন্সী কেয়ার বট। গর্ভাবস্থায় যেকোনো জিজ্ঞাসা আমাকে করতে পারেন। আপনাকে কিভাবে সাহায্য করতে পারি?")
qstn = input()
print(f"Human: {qstn}")
response = rag_chain.invoke(qstn)
ans = response.split("।।")[0]
print(f"Bot:{ans}")

Bot: আমি প্রেগনেন্সী কেয়ার বট। গর্ভাবস্থায় যেকোনো জিজ্ঞাসা আমাকে করতে পারেন। আপনাকে কিভাবে সাহায্য করতে পারি?
Human: প্রেগনেন্সী টেস্ট কিভাবে করবো?
Bot: প্রেগন্যান্সি টেস্ট করার জন্য আপনার নিকটস্থ ফার্মেসি বা সুপারশপ থেকে প্রেগন্যান্সি টেস্ট কিট/বক্স কিনে নিন। দিনের যেকোনো সময়ের প্রস্রাবের নমুনা নিয়ে প্রেগন্যান্সি টেস্ট করা যায়। প্রেগন্যান্সি টেস্ট কিট/স্টিকের উপর প্রস্রাব করার পর সাধারণত কয়েক মিনিটের মধ্যেই ফলাফল দেখায়। 


In [18]:
response = rag_chain.invoke("গর্ভবতী হলে হরমোন খেতে হয়?")
print(response.split("।।")[0])

 হরমোন খেতে হয় না, গর্ভবতী হলে শরীরে নিজেই হরমোন তৈরি হয়


In [87]:
print("Bot: আমি প্রেগনেন্সী কেয়ার বট। গর্ভাবস্থায় যেকোনো জিজ্ঞাসা আমাকে করতে পারেন। আপনাকে কিভাবে সাহায্য করতে পারি?")
qstn = input()
print(f"Human: {qstn}")
response = rag_chain.invoke(qstn)
ans = response.split("।।")[0]
print(f"Bot:{ans}")

Bot: আমি প্রেগনেন্সী কেয়ার বট। গর্ভাবস্থায় যেকোনো জিজ্ঞাসা আমাকে করতে পারেন। আপনাকে কিভাবে সাহায্য করতে পারি?
Human: গর্ভাবস্থায় কি কি হরমোন প্রভাব রাখে?
Bot: গর্ভাবস্থায় হিউম্যান কোরিওনিক গোনাডোট্রোপিন হরমোন, এস্ট্রোজেন হরমোন, প্রোজেস্টেরোন হরমোন ইত্যাদি হরমোন প্রভাব রাখে


# Q/A

In [90]:
response = rag_chain.invoke("প্রেগনেন্সী টেস্ট কিভাবে করবো?")
print(response.split("।।")[0])

 প্রেগন্যান্সি টেস্ট কিট কেনা হয়। আগামী সপ্তাহে নির্ধারিত সময়ে মাসিক না হলে আপনি হয়তো চিন্তা করবেন গর্ভধারণ হলো কি না। তখন পরীক্ষা করার জন্য প্রেগন্যান্সি টেস্ট কিনে রাখতে পারেন। প্রেগন্যান্সি টেস্টের মাধ্যমে আপনি গর্ভধারণ করেছেন কি না সেটি নির্ধারণ করা যাবে। 


In [89]:
response = rag_chain.invoke("মাসিকের সাথে গর্ভবতী হওয়ার সম্পর্ক কি?")
print(response.split("।।")[0])

 মাসিক শুরু হওয়ার আগেই আবারও গর্ভধারণ সম্ভব। তাই পিরিয়ড হয়নি বলে এখন গর্ভধারণ করার সম্ভাবনা নেই—এটা ভেবে ভুল করবেন না


In [88]:
for chunk in rag_chain.stream("প্রেগনেন্ট হলে কী খাওয়া দরকার?"):
    print(chunk, end="", flush=False)

 প্রেগনেন্ট হলে আপনাকে প্রতিদিন ছয় ধরনের খাবার অবশ্যই খেতে হবে। এই ছয় ধরনের খাবার হলো—আমলকি, আমড়া, জাম, জলপাই, লেবু, জাম্বুরা, কমলা ও মাল্টা। এগুলোতে ভিটামিন সি থাকে। ভিটামিন সি আপনার ও গর্ভের শিশুর ত্বক, রক্তনালী ও হাড় সুস্থ রাখতে সাহায্য করবে।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।

KeyboardInterrupt: 

In [64]:
print(rag_chain.invoke("প্রেগনেন্ট হলে কি কি খাওয়া যায়?"))

 প্রেগনেন্ট হলে আপনাকে প্রতিদিন ছয় ধরনের খাবার অবশ্যই খেতে হবে। এই ছয় ধরনের খাবার হলো— ফাইবার সমৃদ্ধ শর্করা, স্বাস্থ্যকর তেলযুক্ত খাবার, প্রোটিন সমৃদ্ধ খাবার, ক্যালসিয়াম সমৃদ্ধ খাবার, ভিটামিন সি সমৃদ্ধ ফল, ও স্বাস্থ্যকর পানীয়।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।।


https://python.langchain.com/docs/tutorials/rag/